In [1]:
import os
import glob
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from matplotlib import pyplot as plt



In [2]:
def conv_bn(x, filters):
    x = layers.Conv1D(filters, kernel_size=1, padding="valid")(x)
    x = layers.BatchNormalization(momentum=0.0)(x)
    return layers.Activation("relu")(x)


def dense_bn(x, filters):
    x = layers.Dense(filters)(x)
    x = layers.BatchNormalization(momentum=0.0)(x)
    return layers.Activation("relu")(x)

class OrthogonalRegularizer(keras.regularizers.Regularizer):
    def __init__(self, num_features, l2reg=0.001):
        self.num_features = num_features
        self.l2reg = l2reg
        self.eye = tf.eye(num_features)

    def __call__(self, x):
        x = tf.reshape(x, (-1, self.num_features, self.num_features))
        xxt = tf.tensordot(x, x, axes=(2, 2))
        xxt = tf.reshape(xxt, (-1, self.num_features, self.num_features))
        return tf.reduce_sum(self.l2reg * tf.square(xxt - self.eye))

def tnet(inputs, num_features):

    # Initalise bias as the indentity matrix
    bias = keras.initializers.Constant(np.eye(num_features).flatten())
    reg = OrthogonalRegularizer(num_features)

    x = conv_bn(inputs, 32)
    x = conv_bn(x, 64)
    x = conv_bn(x, 512)
    x = layers.GlobalMaxPooling1D()(x)
    x = dense_bn(x, 256)
    x = dense_bn(x, 128)
    x = layers.Dense(
        num_features * num_features,
        kernel_initializer="zeros",
        bias_initializer=bias,
        activity_regularizer=reg,
    )(x)
    feat_T = layers.Reshape((num_features, num_features))(x)
    # Apply affine transformation to input features
    return layers.Dot(axes=(2, 1))([inputs, feat_T])

In [3]:
NUM_POINTS = 2048
NUM_CLASSES = 4
inputs_x = keras.Input(shape=(NUM_POINTS, 3))

x = tnet(inputs_x, 3)
x = conv_bn(x, 32)
x = conv_bn(x, 32)
x = tnet(x, 32)
x = conv_bn(x, 32)
x = conv_bn(x, 64)
x = conv_bn(x, 512)
x = layers.GlobalMaxPooling1D()(x)
x = dense_bn(x, 256)
x = layers.Dropout(0.3)(x)
x = dense_bn(x, 128)
x = layers.Dropout(0.3)(x)

outputs = layers.Dense(NUM_CLASSES, activation="softmax")(x)

model = keras.Model(inputs=inputs_x, outputs=outputs, name="pointnet")
model.summary()

Model: "pointnet"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 2048, 3)]    0                                            
__________________________________________________________________________________________________
conv1d (Conv1D)                 (None, 2048, 32)     128         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 2048, 32)     128         conv1d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 2048, 32)     0           batch_normalization[0][0]        
___________________________________________________________________________________________

In [8]:
classification = model.load_weights('/data/home/zhaoyj/mor_classification_parameters.h5')

In [9]:
test_points = []
test_labels = []
folders_test = os.listdir('/data/home/zhaoyj/brain/mor_test')
count = 0
for folder in folders_test:
    folder_path = os.path.join('/data/home/zhaoyj/brain/mor_test',folder)
    files = os.listdir(folder_path)
    for file in files:
        file_path = os.path.join(folder_path,file)
        test_labels.append(count)
        tmp = np.loadtxt(file_path)
        print(file_path)
        tmp = tmp.tolist()
        test_points.append(tmp)        
    count += 1

/data/home/zhaoyj/brain/mor_test/3/res_f_0.032_u08.txt
/data/home/zhaoyj/brain/mor_test/3/res_f_0.032_u11.txt
/data/home/zhaoyj/brain/mor_test/3/res_f_0.032_u0875.txt
/data/home/zhaoyj/brain/mor_test/3/res_f_0.032_u0775.txt
/data/home/zhaoyj/brain/mor_test/3/res_f_0.032_u095.txt
/data/home/zhaoyj/brain/mor_test/1/res_f_0.008_u1.txt
/data/home/zhaoyj/brain/mor_test/1/res_f_0.008_u105.txt
/data/home/zhaoyj/brain/mor_test/1/res_f_0.008_u0925.txt
/data/home/zhaoyj/brain/mor_test/1/res_f_0.008_u11.txt
/data/home/zhaoyj/brain/mor_test/2/res_f_0.042_u0925.txt
/data/home/zhaoyj/brain/mor_test/2/e_0.040_u1.txt
/data/home/zhaoyj/brain/mor_test/2/res_f_0.042_u11.txt
/data/home/zhaoyj/brain/mor_test/2/res_f_0.042_u09.txt
/data/home/zhaoyj/brain/mor_test/2/res_f_0.042_u075.txt
/data/home/zhaoyj/brain/mor_test/2/res_f_0.042_u105.txt
/data/home/zhaoyj/brain/mor_test/2/res_f_0.043_u09.txt
/data/home/zhaoyj/brain/mor_test/0/res_f_0.072_u0825.txt
/data/home/zhaoyj/brain/mor_test/0/res_f_0.072_u095.txt
/

In [10]:
test_points_array = np.array(test_points)

In [11]:
results = model.predict(test_points_array)

In [12]:
results

array([[9.57666934e-01, 1.03735039e-02, 3.56099010e-03, 2.83985417e-02],
       [9.97257948e-01, 9.22661566e-04, 1.04286172e-03, 7.76572619e-04],
       [9.78006840e-01, 6.87516434e-03, 3.17498809e-03, 1.19429892e-02],
       [9.97694075e-01, 6.69957022e-04, 8.63590743e-04, 7.72312575e-04],
       [9.89660382e-01, 3.33415228e-03, 2.22452427e-03, 4.78098355e-03],
       [6.07623224e-05, 9.99868631e-01, 1.32560317e-05, 5.72821446e-05],
       [6.34040480e-05, 9.99863863e-01, 1.38351252e-05, 5.89002193e-05],
       [5.38852437e-05, 9.99881268e-01, 1.17874406e-05, 5.30711368e-05],
       [6.68508874e-05, 9.99857664e-01, 1.46022385e-05, 6.08228802e-05],
       [2.24713804e-04, 4.62270255e-04, 9.98646200e-01, 6.66780397e-04],
       [4.64533077e-04, 7.48914841e-04, 9.98096168e-01, 6.90376852e-04],
       [4.57392394e-04, 5.27614204e-04, 9.98302102e-01, 7.12887384e-04],
       [2.05023913e-04, 4.40389122e-04, 9.98753428e-01, 6.01163774e-04],
       [5.33266051e-04, 6.07465976e-04, 9.98169065e

In [13]:
x = np.argmax(results, 1)

In [14]:
x

array([0, 0, 0, 0, 0, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 3, 3, 0, 3, 3, 3])

In [15]:
y=np.array(test_labels)

In [16]:
y

array([0, 0, 0, 0, 0, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3])